### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water and 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

### Answer
#### 1.
Calculate saturation pressures:
$
\begin{aligned}
p_{1,4 dioxane}^{sat} = 10^{ a_1 - \frac{a_2}{T + a_3}} \\
p_{water}^{sat} = 10^{ a_1 - \frac{a_2}{T + a_3}} \\
\end{aligned}
$

Then p can be calculated as:

$
\begin{aligned}
\hat{p} = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}
+ x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}\\
= & x_1\exp\left(A_{12}\left(\frac{A_{21}(1-x_1)}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)10^{ 8.07131 - \frac{1730.63}{T + 233.426}} 
+ (1-x_1)\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)10^{ 7.43155 - \frac{1554.679}{T + 240.337}}\\
\end{aligned}
$

The least square problem can be fomulated as:

$
\begin{aligned}
\min_{A_{12}, A{21}} \text{loss}  = \sum \left(\hat{p}-p\right)^{2}
= \sum\left(x_1\exp\left(A_{12}\left(\frac{A_{21}(1-x_1)}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)10^{ a_1 - \frac{a_2}{T + a_3}} 
+ (1-x_1)\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}(1-x_1)}\right)^2\right)10^{ a_1 - \frac{a_2}{T + a_3}}-p\right)^{2}\\
\end{aligned}
$

#### 2.

In [29]:
import numpy as np
import torch as t
from torch.autograd import Variable

x1_data = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
p_data = [28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]
p_water = 10.0**(8.07131-1730.63/(20.0+233.426))
p_dio = 10.0**(7.43155-1554.679/(20.0+240.337))

def loss(a): 
    loss_value = 0.0
    for i in range(10):
        x1 = x1_data[i]
        p = p_data[i]
        p_fit = x1*p_water*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2)+(1-x1)*p_dio*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2)
        print(p_fit)
        loss_value = loss_value + (p_fit-p)**2
    return loss_value
A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)
loss(A).backward()
err = t.linalg.norm(A.grad) # calculate error

n = 0.1 # searching speed
# iteration until error < eps
while err >= 0.1:
    loss(A).backward()
    A -= 
    print(A)
    err = t.linalg.norm(A.grad)


tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
ten

tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor

tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor

tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor

tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78

...

tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor

tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78

tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor

tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor(20.5492, grad_fn=<AddBackward0>)
tensor(18.7954, grad_fn=<AddBackward0>)
tensor([1., 1.], requires_grad=True)
tensor(78.3520, grad_fn=<AddBackward0>)
tensor(62.2424, grad_fn=<AddBackward0>)
tensor(50.3590, grad_fn=<AddBackward0>)
tensor(41.4916, grad_fn=<AddBackward0>)
tensor(34.8066, grad_fn=<AddBackward0>)
tensor(29.7235, grad_fn=<AddBackward0>)
tensor(25.8332, grad_fn=<AddBackward0>)
tensor(22.8447, grad_fn=<AddBackward0>)
tensor

KeyboardInterrupt: 

In [12]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Define a loss
loss = (x[0] - 1)**2 + (x[1] - 2)**2

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
x.grad.numpy()

array([ 0., -4.], dtype=float32)

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).

### Answer

In [2]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Define a loss
loss = (x[0] - 1)**2 + (x[1] - 2)**2

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
x.grad.numpy()

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Let's examine the gradient at a different x.
x.data = t.tensor([2.0, 1.0])
loss = (x[0] - 1)**2 + (x[1] - 2)**2
loss.backward()
x.grad.numpy()

In [ ]:
# Here is a code for gradient descent without line search

import torch as t
from torch.autograd import Variable

x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Fix the step size
a = 0.01

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = (x[0] - 1)**2 + (x[1] - 2)**2
    loss.backward()
    
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        x -= a * x.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.grad.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())